# Build Word2Vec

## Helpers

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# coding=utf-8
# https://github.com/motazsaad/process-arabic-text/blob/master/clean_arabic_text.py
import re
import string
import sys
import nltk
from nltk.corpus import stopwords
  
nltk.download('stopwords')
nltk.download('punkt')
stopwords_arabic = stopwords.words("arabic")

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ»«'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("ب ال", "بال", text) #me
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)


word_clean_limit = 5
#me
#https://r12a.github.io/app-conversion/
def remove_arabic_additions(sentence):
    sentence_clean = ""

    for text in nltk.word_tokenize(sentence):
        if len(text) < word_clean_limit:
            text = " " + text +  " "
            sentence_clean += text 
            continue

        text = " " + text +  " "
        #بال
        #بالموضوعات
        hah = u'\u0020\u0628\u0627\u0644'
        space_hah = u'\u0020\u0628 \u0627\u0644\u0020'
        text = re.sub(hah,space_hah, text)

        #فال
        #فالموضوعات
        hah = u'\u0020\u0641\u0627\u0644'
        space_hah = u'\u0020\u0641 \u0627\u0644\u0020'
        text = re.sub(hah,space_hah, text)

        #وال
        #والموضوعات
        #(covered in clean text , but i need this function to be indepnedant)
        hah = u'\u0020\u0648\u0627\u0644'
        space_hah = u'\u0020\u0648 \u0627\u0644\u0020'
        text = re.sub(hah,space_hah, text)

        text = text.replace(' ال', ' ال ')

        #ات
        #موضوعات
        hah = u'\u0627\u062A\u0020'
        space_hah = u'\u0020\u0627\u062A\u0020'
        text = re.sub(hah,space_hah, text)
    
        #ان
        #موضوعان
        hah = u'[^\u0020]\u0627\u0646\u0020'
        space_hah = u'\u0020\u0627\u0646\u0020'
        text = re.sub(hah,space_hah, text)

        #ها
        #كتابها
        hah = u'\u0647\u0627\u0020'
        space_hah = u'\u0020\u0647\u0627\u0020'
        text = re.sub(hah,space_hah, text)

        #هم
        #كتابهم
        hah = u'\u0647\u0645\u0020'
        space_hah = u'\u0020\u0647\u0645\u0020'
        text = re.sub(hah,space_hah, text)

        #نا
        #كتابنا
        hah = u'\u0646\u0627\u0020'
        space_hah = u'\u0020\u0646\u0627\u0020'
        text = re.sub(hah,space_hah, text)

        #ين
        #لاعبين
        hah = u'\u064A\u0646\u0020'
        space_hah = u'\u0020\u064A\u0646\u0020'
        text = re.sub(hah,space_hah, text)
    
        #ون
        #لاعبون
        hah = u'\u0648\u0646\u0020'
        space_hah = u'\u0020\u0648\u0646\u0020'
        text = re.sub(hah,space_hah, text)

        #يا
        #مصريا
        hah = u'\u064A\u0627\u0020'
        space_hah = u'\u0020\u064A\u0627\u0020'
        text = re.sub(hah,space_hah, text)
    
        #كم
        #كتنبكم
        hah = u'\u0643\u0645\u0020'
        space_hah = u'\u0020\u0643\u0645\u0020'
        text = re.sub(hah,space_hah, text)

        #كن
        #كتنبكن
        hah = u'\u0643\u0646\u0020'
        space_hah = u'\u0020\u0643\u0646\u0020'
        text = re.sub(hah,space_hah, text)

        sentence_clean += text 
    sentence_clean = sentence_clean.replace("  "," ")
    return sentence_clean
    
 
  
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    #trim    
    text = text.strip()
    
    text = remove_punctuations(text)
    text = remove_diacritics(text)
    #text = remove_repeating_char(text)
    
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?'  ,'؟'  ,'!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و "," يا ","","",""      ,"ي",""  ,' ', ' ' ,' '     ,' ? ',' ? ',' ! ']
    
    
    search_wow = ['وا','ول']
    replace_wow = ['و ا','و ل']
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    for i in range(0, len(search_wow)):
        text = text.replace(search_wow[i], replace_wow[i])
       
    text = text.replace(".", "")

    # remove numbers
    text = ''.join([i for i in text if not i.isdigit()])
    
    #####text = remove_arabic_additions(text)

    # Optionally, remove stop words
    #if remove_stopwords:
    #    text = text.split()
    #    stops = set(stopwords_arabic)
    #    text = [w for w in text if not w in stops]
    #    text = " ".join(text)
    
    return text


import pickle
def save(obj , filename):
  print("saving {} ..".format(filename))
  with open(filename, 'wb') as handle:
      pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
      
def load(filename):
  print("loading {} ..".format(filename))
  with open(filename, 'rb') as handle:
    return pickle.load(handle)


    
#aaa = clean_text("""تقدم لكم موقع "الدستور" الإلكتروني بثًا مباشرًا لمباراة مانشستر يونايتد أمام ستوك سيتي، المقرر لها بعد قليل على ملعب "أولد ترافورد" ضمن منافسات الجولة السابعة، من بطولة الدوري الإنجليزي الممتاز الذين جون الله "البريميرليج".
#""")
#dsd = remove_arabic_additions(" والموضوعات  الدوري"  )
#bbb = remove_arabic_additions(" والموضوعات  "  )
#
#sdfb = nltk.word_tokenize(bbb)
#pass



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from __future__ import print_function


import json
import os
import pandas
import io
import sys
import re


class ProgressBar(object):
    DEFAULT = 'Progress: %(bar)s %(percent)3d%%'
    FULL = '%(bar)s %(current)d/%(total)d (%(percent)3d%%) %(remaining)d to go'

    def __init__(self, total, width=40, fmt=DEFAULT, symbol='=',
                 output=sys.stderr):
        assert len(symbol) == 1

        self.total = total
        self.width = width
        self.symbol = symbol
        self.output = output
        self.fmt = re.sub(r'(?P<name>%\(.+?\))d',
            r'\g<name>%dd' % len(str(total)), fmt)

        self.current = 0

    def __call__(self):
        percent = self.current / float(self.total)
        size = int(self.width * percent)
        remaining = self.total - self.current
        bar = '[' + self.symbol * size + ' ' * (self.width - size) + ']'

        args = {
            'total': self.total,
            'bar': bar,
            'current': self.current,
            'percent': percent * 100,
            'remaining': remaining
        }
        print('\r' + self.fmt % args, file=self.output, end='')

    def done(self):
        self.current = self.total
        self()
        print('', file=self.output)

In [0]:
default_path = "drive/Colab Notebooks/Model 4_5/"

#-------------------------------save/load--------------------------------------#
pickle_path = default_path + "pickles/"

import pickle

def save(obj , filename):
  print("saving {} ..".format(filename))
  with open(filename, 'wb') as handle:
      pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
      
def load(filename):
  print("loading {} ..".format(filename))
  with open(filename, 'rb') as handle:
    return pickle.load(handle)
#-----------------------------------------------------------------------------------#  

## Train

In [0]:
default_path = "drive/Colab Notebooks/Model 4_5/"
reviews_csv = default_path + "ArabicMogalad.csv"

In [0]:
import pandas as pd
reviews = pd.read_csv(reviews_csv)
reviews.head()

,Unnamed: 0,Text,Summary
0,0,"تقدم لكم موقع ""الدستور"" الإلكتروني بثًا مباشرً...","بث مباشر ""بدون تقطيع"" لمباراة مانشستر يونايتد ..."
1,1,"بعثة المنتخب المغربي تحل بـ ""مالابو "" بلاَعِبَ...","بعثة المنتخب المغربي تحل بـ ""مالابو "" بلاَعِبَ..."
2,2,رقم المشاركة : [ 1 ] تساعد الملوخية على تهدئة ...,الملوخية ..لذيذة وصحية
3,3,شرح محمد إبراهيم مدرب عام فريق بتروجيت، الوسيل...,"مدرب بتروجيت يشرح لـ""الدستور"" كيف حافظ فريقه ع..."
4,4,والسؤال: كيفّ يضع إبليس عرشه على الماء ؟ شاهد ...,هل تعلم أين يعيش أبليس و ما هي علاقته بمثلث بر...


In [0]:
documents = reviews["Text"] 
summary = reviews["Summary"]

In [0]:
print(len(documents))
print(len(summary))

265476
265476


In [0]:
clean_documents = []
print("loading documents...")
progress = ProgressBar(len(documents), fmt=ProgressBar.FULL)
for doc in documents:
  clean_documents.append(clean_text(doc))
  progress.current += 1
  progress()
progress.done()
save(clean_documents , "clean_documents_ketab.pkl")


clean_summary = []
print("loading summaries...")
progress = ProgressBar(len(summary), fmt=ProgressBar.FULL)
for doc in summary:
  clean_summary.append(clean_text(doc))
  progress.current += 1
  progress()
progress.done()
save(clean_summary , "clean_summary_ketab.pkl")

loading documents...


[========================================] 265476/265476 (100%)      0 to go


saving clean_documents_ketab.pkl ..
loading summaries...


[========================================] 265476/265476 (100%)      0 to go


saving clean_summary_ketab.pkl ..


In [0]:
print(clean_documents[0])

تقدم موقع الدستور الالكتروني بثا مباشرا لمباراه مانشستر يونايتد امام ستوك سيتي المقرر قليل علي ملعب او لد ترافورد ضمن منافسات الجو السابعه بطو الدوري الانجليزي الممتاز البريميرليج للمشاهده اضغط


In [0]:
default_path

'drive/Colab Notebooks/Model 4_5/'

In [0]:
clean_documents_list = clean_documents + clean_summary

In [0]:
clean_documents_list[0]

'تقدم لكم موقع الدستور الالكتروني بثا مباشرا لمباراه مانشستر يونايتد امام ستوك سيتي المقرر لها بعد قليل علي ملعب او لد ترافورد ضمن منافسات الجو له السابعه من بطو له الدوري الانجليزي الممتاز البريميرليج\r\r \r\r للمشاهده اضغط هنا'

In [0]:
len(clean_documents_list)

530952

In [7]:
import pandas as pd

reviews = pd.read_csv("drive/My Drive/Colab Notebooks/Model 4_5/ArabicMogalad_Ndeef_Extreme.csv")
reviews.shape
reviews.isnull().sum()
reviews = reviews.filter(['Text', 'Summary'])
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)
reviews.head()

,Text,Summary
0,تقدم لكم موقع ال دستور ال الكتروني بثا مباشرا...,بث مباشر بدون تقطيع لمباراه مانشستر يونايتد و...
1,بعثه ال منتخب ال مغربي تحل ب مالابو بلاعب ين ...,بعثه ال منتخب ال مغربي تحل ب مالابو بلاعب ين ...
2,رقم ال مشاركه تساعد ال ملوخيه علي تهدئه ال اع...,ال ملوخيه لذيذه وصحيه
3,شرح محمد ابراهيم مدرب عام فريق بتروجيت ال وسي...,مدرب بتروجيت يشرح لالدستور كيف حافظ فريقه علي...
4,و ال سؤال كيف يضع ابليس عرشه علي الماء شاهد ا...,هل تعلم اين يعيش ابليس و ما هي علاقته بمثلث ب...


In [10]:
clean_documents_list = []
progress = ProgressBar(len(reviews.Text + reviews.Summary), fmt=ProgressBar.FULL)
for d in reviews.Text + reviews.Summary:
  clean_documents_list.append(d.split())
  progress.current += 1
  progress()
progress.done()

[========================================] 264790/264790 (100%)      0 to go


In [14]:
(reviews.Text + reviews.Summary)[0]

' تقدم لكم موقع ال دستور ال الكتروني بثا مباشرا لمباراه مانشستر يونايتد امام ستوك سيتي ال مقرر لها بعد قليل علي ملعب او لد ترافورد ضمن منافس ات الجو له ال سابعه من بطو له ال دوري ال انجليزي ال ممتاز ال بريميرليج للمشاهده اضغط هنا  بث مباشر بدون تقطيع لمباراه مانشستر يونايتد وستوك سيتي في ال بريميرليج '

In [16]:
(reviews.Summary)[0]

' بث مباشر بدون تقطيع لمباراه مانشستر يونايتد وستوك سيتي في ال بريميرليج '

In [17]:
import gensim
model_arabic_vec = gensim.models.Word2Vec(
        clean_documents_list,
        size=150,
        window=10,
        min_count=2,
        workers=10)
model_arabic_vec.train(clean_documents_list, total_examples=len(clean_documents_list), epochs=10)
model_arabic_vec.wv.save(default_path +"model_arabic_extreme.model")

FileNotFoundError: ignored

In [19]:
default_path

'drive/Colab Notebooks/Model 4_5/'

In [20]:
model_arabic_vec.wv.save('drive/My Drive/Colab Notebooks/Model 4_5/' +"model_arabic_extreme.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [18]:
model_arabic_vec.wv["كتاب"]

array([ 0.60049486,  0.66773444,  0.8743169 , -1.0486274 ,  0.64036316,
       -0.6696772 , -1.19206   ,  2.0246813 ,  0.43678576,  4.697042  ,
       -0.3702323 , -2.0760772 ,  1.8011181 , -1.3115736 , -0.56840664,
        2.935423  , -5.3931212 , -0.30702448,  2.296707  ,  1.6368376 ,
        0.7924471 , -0.683585  ,  1.0715748 ,  0.47236156, -4.9426765 ,
       -3.3378832 , -3.7441792 ,  2.0538943 ,  3.573936  ,  4.348934  ,
       -0.5823462 ,  2.6171596 , -0.48603222, -1.2909033 ,  0.9464881 ,
        1.1155788 , -2.439662  , -3.8835826 ,  1.8998314 , -0.98640794,
       -1.8975354 , -0.52067894,  1.777562  ,  3.3140233 , -1.3278078 ,
       -0.650048  ,  2.8352718 , -1.1413844 , -1.6535718 ,  1.799087  ,
       -7.2287045 , -1.7522964 , -6.6696224 ,  4.2940865 ,  2.913149  ,
       -1.7206786 , -3.66025   ,  1.0703232 ,  4.0335255 ,  2.2195156 ,
       -0.02473911,  2.3235254 ,  0.0451945 ,  3.3627374 , -1.3588668 ,
        1.0655632 ,  1.0160086 , -0.369204  ,  0.5264064 ,  2.22

In [22]:
w1 = "ال"
model_arabic_vec.wv.most_similar(positive = w1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('و', 0.5625959634780884),
 ('ات', 0.5462248921394348),
 ('لامعياريه', 0.5217580199241638),
 ('ايوي', 0.4790849983692169),
 ('تفتيس', 0.47427698969841003),
 ('Docs', 0.47099539637565613),
 ('muslim', 0.46997708082199097),
 ('ب', 0.4641309678554535),
 ('زوغت', 0.4564330279827118),
 ('عاطفيهيمتلك', 0.44359564781188965)]

In [0]:
model.save('model_arabic_vec.bin')

In [0]:
model_arabic_vec.wv.save(default_path +"model_arabic_vec.model")

In [0]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load(default_path +"model_arabic_ketab.model", mmap='r')

In [0]:
wv.most_similar(positive = "خبر")

KeyError: ignored

In [0]:
wv["سيسي"]